# Evolving Emission Measure Distributions
Look at the emission measure distribution ($\mathrm{EM}(T)$) as a function of time for a given run, for both a uniform and power-law case, comparing equilibrium and non-equilibrium results. Choose an intermediate frequency, say around $t_N\approx2000$ s to show some interesting results.

In [50]:
import os,sys
sys.path.append(os.path.join(os.environ['RESEARCH_DIR'],'EBTEL_analysis/src'))
from random import randint
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import animation
import seaborn.apionly as sns
import em_binner as emb
%matplotlib inline

## Data Selection
Grab the data we want from our database. For the power-law case, we'll start off with a dataset where the waiting time is **not** proportional to the event energy.

In [51]:
em_results_dict = {}
tn_sample_val = 2500
#uniform
data = np.loadtxt('/data/datadrive2/EBTEL_runs/electron_heating_runs/alphauniform/data/ebtel_L40.0_tn%d_tpulse200.0_rka4/ebtel_L40.0_tn%d_tpulse200.0_rka4_0.txt'%(tn_sample_val,tn_sample_val))
em_results_dict['uniform_ieq'] = {'t':data[:,0],'T':data[:,1],'n':data[:,3],'q':data[:,15]}
data = np.loadtxt('/data/datadrive2/IonPopSolver_runs/electron_heating_runs/alphauniform/data/ebtel_L40.0_tn%d_tpulse200.0_rka4/ebtel_L40.0_tn%d_tpulse200.0_rka4_0.ips_results.txt'%(tn_sample_val,tn_sample_val))
em_results_dict['uniform_nei'] = {'t':data[:,0],'T':data[:,2],'n':data[:,3]}
#power-law
data = np.loadtxt('/data/datadrive2/EBTEL_runs/electron_heating_runs/alpha2.5/data/ebtel_L40.0_tn%d_tpulse200.0_rka4/ebtel_L40.0_tn%d_tpulse200.0_rka4_0.txt'%(tn_sample_val,tn_sample_val))
em_results_dict['pl_ieq'] = {'t':data[:,0],'T':data[:,1],'n':data[:,3],'q':data[:,15]}
data = np.loadtxt('/data/datadrive2/IonPopSolver_runs/electron_heating_runs/alpha2.5/data/ebtel_L40.0_tn%d_tpulse200.0_rka4/ebtel_L40.0_tn%d_tpulse200.0_rka4_0.ips_results.txt'%(tn_sample_val,tn_sample_val))
em_results_dict['pl_nei'] = {'t':data[:,0],'T':data[:,2],'n':data[:,3]}

## Emission Measure Calculation
Calculate the emission measure at each timestep for each of the datasets.

In [52]:
em_maker = emb.EM_Binner(2.*40.0e+8)

In [53]:
for key in em_results_dict:
    em_maker.set_data(em_results_dict[key]['t'],em_results_dict[key]['T'],em_results_dict[key]['n'])
    em_maker.build_em_dist(build_mat=True)
    em_results_dict[key]['em_mat'] = em_maker.em_mat
    em_results_dict[key]['T_em'] = em_maker.T_em

## Plotting $\mathrm{EM}(T)$ versus time, $t$
Best way to do this is with a movie. First, make a function for making the movie.

In [54]:
#stuff to make animation inline

from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

from IPython.display import HTML

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

In [61]:
def make_em_movie(ieq_data,nei_data, interval=20, fps=30, movie_filename=None):
    """Make movie of evolving EM distribution"""
    
    #set up the figure
    fig = plt.figure(figsize=(12,8))
    gs = gridspec.GridSpec(3,6)
    ax_heat = fig.add_subplot(gs[0,:])
    ax_snap = fig.add_subplot(gs[1:,0:2])
    ax_ieq = fig.add_subplot(gs[1:,2:4])
    ax_nei = fig.add_subplot(gs[1:,4:])
    
    #set limits
    ax_heat.set_xlim([ieq_data['t'][0],ieq_data['t'][-1]])
    ax_heat.set_ylim([0,np.max(ieq_data['q'])])
    ax_snap.set_xlim([10**5.5,10**7.5])
    ax_snap.set_ylim([10**26.,10**29.])
    ax_ieq.set_xlim([10**5.5,10**7.5])
    ax_ieq.set_ylim([10**26.,10**29.])
    ax_nei.set_xlim([10**5.5,10**7.5])
    ax_nei.set_ylim([10**26.,10**29.])
    
    #labels and ticks and scale
    ax_heat.set_xlabel(r'$t\,\,\mathrm{(s)}$',fontsize=22)
    ax_heat.set_ylabel(r'$H\,\,(\mathrm{erg}\,\mathrm{cm}^{-3}\,\mathrm{s}^{-1})$',fontsize=22)
    ax_snap.set_xlabel(r'$T\,\,\mathrm{(K)}$',fontsize=22)
    ax_ieq.set_xlabel(r'$T\,\,\mathrm{(K)}$',fontsize=22)
    ax_nei.set_xlabel(r'$T\,\,\mathrm{(K)}$',fontsize=22)
    ax_snap.set_ylabel(r'$\mathrm{EM}\,\,(\mathrm{cm}^{-5})$',fontsize=22)
    ax_ieq.tick_params(axis='y',labelleft='off')
    ax_nei.tick_params(axis='y',labelleft='off')
    ax_snap.set_yscale('log')
    ax_snap.set_xscale('log')
    ax_ieq.set_yscale('log')
    ax_ieq.set_xscale('log')
    ax_nei.set_yscale('log')
    ax_nei.set_xscale('log')
    
    #plot the heating
    ax_heat.plot(ieq_data['t'],ieq_data['q'],linewidth=2,color=sns.color_palette('deep')[0])
    
    plt.tight_layout()

    #setup the plotters
    progress, = ax_heat.plot([],[],linewidth=2,color='k',linestyle='--')
    em_ieq_snap, = ax_snap.plot([],[],linestyle='-',color=sns.color_palette('deep')[0],linewidth=2)
    em_nei_snap, = ax_snap.plot([],[],linestyle='--',color=sns.color_palette('deep')[1],linewidth=2)
    em_snap_ieq, = ax_ieq.plot([],[],linestyle='-',color=sns.color_palette('deep')[0],linewidth=2)
    em_avg_ieq, = ax_ieq.plot([],[],linestyle='--',color=sns.color_palette('deep')[1],linewidth=2)
    em_snap_nei, = ax_nei.plot([],[],linestyle='-',color=sns.color_palette('deep')[0],linewidth=2)
    em_avg_nei, = ax_nei.plot([],[],linestyle='--',color=sns.color_palette('deep')[1],linewidth=2)
    
    #define initializer
    def init():
        progress.set_data([],[])
        em_ieq_snap.set_data([],[])
        em_nei_snap.set_data([],[])
        em_snap_ieq.set_data([],[])
        em_avg_ieq.set_data([],[])
        em_snap_nei.set_data([],[])
        em_avg_nei.set_data([],[])
        return em_ieq_snap,em_nei_snap,em_snap_ieq,em_avg_ieq,em_snap_nei,em_avg_nei
    
    #define animator function
    def animate(i):
        #get data and compute running averages
        em_i = ieq_data['em_mat'][i,:]
        em_n = nei_data['em_mat'][i,:]
        if i > 0:
            em_i_ra = np.average(ieq_data['em_mat'][0:i+1,:],weights=np.gradient(ieq_data['t'][0:i+1]),axis=0)
            em_n_ra = np.average(nei_data['em_mat'][0:i+1,:],weights=np.gradient(nei_data['t'][0:i+1]),axis=0)
        else:
            em_i_ra = em_i
            em_n_ra = em_n
        #set data
        progress.set_data([ieq_data['t'][i],ieq_data['t'][i]],[0,np.max(ieq_data['q'])])
        em_ieq_snap.set_data(ieq_data['T_em'],em_i)
        em_nei_snap.set_data(nei_data['T_em'],em_n)
        em_snap_ieq.set_data(ieq_data['T_em'],em_i)
        em_avg_ieq.set_data(ieq_data['T_em'],em_i_ra)
        em_snap_nei.set_data(nei_data['T_em'],em_n)
        em_avg_nei.set_data(nei_data['T_em'],em_n_ra)
        return em_ieq_snap,em_nei_snap,em_snap_ieq,em_avg_ieq,em_snap_nei,em_avg_nei
    
    #make animation
    anim = animation.FuncAnimation(fig,animate,init_func=init, frames=len(ieq_data['t']), blit=True, )

    #save or return animation handler
    if movie_filename is not None:
        metadata={'title':'Evolving EM distributions, IEQ and NEI','author':'Will Barnes'}
        anim.save(movie_filename,writer=animation.writers['ffmpeg'](fps=fps,metadata=metadata,bitrate=1800))
    else:
        plt.close(anim._fig)
        return anim
    

In [62]:
anim=make_em_movie(em_results_dict['uniform_ieq'],em_results_dict['uniform_nei'])

In [63]:
display_animation(anim)

AttributeError: 'bytes' object has no attribute 'encode'